<a href="https://colab.research.google.com/github/HeptaDecane/LP1_SEM7/blob/main/CudaSandbox/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-11-25 21:00:33--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-11-25 21:00:33--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?Uk0-QTvliBgZVZn7c3ddrBeapuVZGMngfA6UZ9xaGQYdZlOoqCUjJDpXj6GctU3T6BpqbyyLe4wGcz4ZvOhdIeWmEkYiw-dAi7aONMqnYpjhe

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


<br/>

In [3]:
!curl https://raw.githubusercontent.com/HeptaDecane/LP1_SEM7/main/CudaSandbox/hello_world.cu --output hello_world.cu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   187  100   187    0     0   1214      0 --:--:-- --:--:-- --:--:--  1214


In [4]:
!nvcc hello_world.cu -o hello_world

In [5]:
!nvprof ./hello_world

Hello World from CPU!
==4370== NVPROF is profiling process 4370, command: ./hello_world
Hello World from GPU!
==4370== Profiling application: ./hello_world
==4370== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  127.65us         1  127.65us  127.65us  127.65us  cuda_hello(void)
      API calls:   99.68%  210.54ms         1  210.54ms  210.54ms  210.54ms  cudaLaunchKernel
                    0.20%  427.18us         1  427.18us  427.18us  427.18us  cuDeviceTotalMem
                    0.11%  222.88us        96  2.3210us     107ns  100.12us  cuDeviceGetAttribute
                    0.01%  26.703us         1  26.703us  26.703us  26.703us  cuDeviceGetName
                    0.00%  6.1620us         1  6.1620us  6.1620us  6.1620us  cuDeviceGetPCIBusId
                    0.00%  1.8820us         3     627ns     229ns     885ns  cuDeviceGetCount
                    0.00%  1.6140us         2     807ns     422ns  1.19

<br/>

In [6]:
!curl https://raw.githubusercontent.com/HeptaDecane/LP1_SEM7/main/CudaSandbox/vector_add.cu --output vector_add.cu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2101  100  2101    0     0  47750      0 --:--:-- --:--:-- --:--:-- 47750


In [7]:
!nvcc vector_add.cu -o vector_add

In [8]:
!nvprof ./vector_add

==4420== NVPROF is profiling process 4420, command: ./vector_add

---------------------------
__SUCCESS__
---------------------------
N                 = 1048576
Threads Per Block = 256
Blocks In Grid    = 4096
---------------------------

==4420== Profiling application: ./vector_add
==4420== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.53%  4.2679ms         1  4.2679ms  4.2679ms  4.2679ms  [CUDA memcpy DtoH]
                   33.58%  2.2560ms         2  1.1280ms  1.1269ms  1.1291ms  [CUDA memcpy HtoD]
                    2.88%  193.54us         1  193.54us  193.54us  193.54us  add_vectors(double*, double*, double*)
      API calls:   94.85%  211.21ms         3  70.404ms  235.05us  210.68ms  cudaMalloc
                    3.58%  7.9772ms         3  2.6591ms  1.0825ms  5.5932ms  cudaMemcpy
                    1.11%  2.4701ms         3  823.37us  229.37us  1.1210ms  cudaFree
                    0.21%  471.83us 